In [ ]:
import sys

# !{sys.executable} -m pip install --upgrade pip
# # uncomment the following lines to install the required packages
# !{sys.executable} -m pip install seaborn
# !{sys.executable} -m pip install gmsh
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install Pyarrow
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install pyvista
# !{sys.executable} -m pip install ipywidgets
# !{sys.executable} -m pip install pyvirtualdisplay



# print("All packages installed!")

## Labels and colours

In [ ]:
# temperature
color_temperature = 'coolwarm'
label_temperature = r'Temperature $T$ [$^\circ$C]'

# gradient
color_gradient = 'Spectral'
label_gradient_partial = r'Gradient $\nabla T$ [$^\circ$C/m]'
label_gradient_partial_x = r'Gradient $\partial T / \partial x$ [$^\circ$C/m]'
label_gradient_partial_y = r'Gradient $\partial T / \partial y$ [$^\circ$C/m]'
label_gradient_g = r'Gradient $\mathbf g$ [$^\circ$C/m]'
label_gradient_g_x = r'Gradient $g_x$ [$^\circ$C/m]'
label_gradient_g_y = r'Gradient $g_y$ [$^\circ$C/m]'

# flux
color_flux = 'seismic'
label_flux = r'Flux $\mathbf q$ [W/m$^2$]'
label_flux_x = r'Flux $q_x$ [W/m$^2$]'
label_flux_y = r'Flux $q_y$ [W/m$^2$]'

# conductivity
color_conductivity = 'viridis'
label_conductivity = r'Thermal conductivity $k$ [W/(m$\cdot$$^\circ$C)]'

# source
color_source_min = 'black'
color_source_max = 'red'
label_source = r'Source term $f$ [W/m$^3$]'


## Imports

In [ ]:
# maths
import numpy as np
import pandas as pd

# plotting graphs settings
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

# sns.set_style("whitegrid")
sns.set_palette("Set2")

# plt.rcParams['figure.figsize'] = [9, 6]
plt.rcParams['figure.figsize'] = [6, 4]
plt.rcParams['figure.dpi'] = 300
# plt.rcParams['font.family'] = "Serif"
plt.rcParams['font.family'] = "Times New Roman"
plt.rcParams['font.size'] = 20

# colour pallete
# plt.rcParams['axes.prop_cycle'] = plt.cycler("color", plt.cm.tab20.colors)

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams['text.usetex'] = True

# Apply the default settings to Seaborn
sns.set_context(rc=plt.rcParams)

# create mesh with gmsh
import gmsh

# plotting vtks
import pyvista as pv
pv.set_plot_theme("document")
import ipywidgets as widgets
from pyvirtualdisplay import Display

print("Imports finished! :D")

In [ ]:
# locations of the executables

# directories
um_view = "$HOME/um_view_release" # change this to the location of your um_view_release or um_view with mofem_data_driven_finite_elements installed
tools_dir = um_view + "/bin"
dd_dir = um_view + "/mofem_data_driven_finite_elements"

# tools
read_med = tools_dir + "/read_med"
mofem_part = tools_dir + "/mofem_part"

# create datasets
create_csv_dataset = dd_dir + "/create_csv_dataset"

# classic diffusion
classic_diffusion = dd_dir + "/classic_diffusion"
hdiv_diffusion = dd_dir + "/hdiv_diffusion"

# data driven diffusion
data_driven_diffusion = dd_dir + "/data_driven_diffusion"
data_driven_diffusion_snes = dd_dir + "/data_driven_diffusion_snes"
hdiv_data_driven_diffusion = dd_dir + "/hdiv_data_driven_diffusion"
hdiv_data_driven_diffusion_snes = dd_dir + "/hdiv_data_driven_diffusion_snes"

# nonlinear diffusion
diffusion_nonlinear_graphite = dd_dir + "/diffusion_nonlinear_graphite"
diffusion_nonlinear_hdiv_graphite = dd_dir + "/diffusion_nonlinear_hdiv_graphite"
diffusion_nonlinear_snes = dd_dir + "/diffusion_nonlinear_snes"

In [ ]:
class AttrDict(dict):
    def __getattr__(self, attr):
        if attr in self:
            return self[attr]
        raise AttributeError(f"'AttrDict' object has no attribute '{attr}'")
    
params = AttrDict() # Attribute dictionary for storing the parameters

In [ ]:
def show_results(params):
    out_to_vtk = !ls -c1 {params.show_file}*vtk
    last_file=out_to_vtk[0]

    mesh = pv.read(last_file[:-3] + "vtk")
    if params.warp_field:
        mesh = mesh.warp_by_vector(vectors=params.warp_field, factor=params.warp_factor)

    if params.show_edges:
        mesh=mesh.shrink(0.95)
    
    jupyter_backend='ipygany'
    cmap = "turbo"

    p = pv.Plotter(notebook=True)
    p.add_mesh(mesh, scalars=params.show_field, component=None, smooth_shading=True, cmap=cmap)
    p.camera_position = "xy"
    
    p.enable_parallel_projection()
    p.enable_image_style()
    
    p.show(jupyter_backend=jupyter_backend)

params.warp_field = ""    # no warp field
params.warp_factor = 1.0  # warp factor
params.show_edges = True # show edges
params.triangle_mesh = True # triangle mesh
params.config_file = "bc.cfg" # config file
params.conductivity = 1.0 # linear conductivity

## Square_top analysis functions

In [ ]:
# generation of a config file - what attributes should the blocksets have
def generateConfig_squareTop(params):
    # Open the file for writing
    with open(params.config_file, 'w') as f:
        # FLUX_SQUARE_TOP boundary condition
        data = ['[SET_ATTR_FLUX_SQUARE_TOP]', 'number_of_attributes=1', 'user1='+str(params.conductivity), ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_UNIFORM boundary condition set to 0
        data = ['[SET_ATTR_PRESSURE_UNIFORM_0]', 'number_of_attributes=1', 'user1=0.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)

In [ ]:
# gmsh creation of a 3D beam + visualisation of it
def generateMesh_squareTop(params):
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)

    square1 = gmsh.model.occ.add_rectangle(0, 0, 0, params.length_x, params.length_y)

    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.occ.synchronize()

    # # ensuring a structured mesh with required element size 
    N = int(params.length_x / params.element_size) + 1

    for n in range(len(gmsh.model.getEntities(1))):
        gmsh.model.mesh.setTransfiniteCurve(n+1, N,'Progression', 1.0)

    gmsh.model.mesh.setTransfiniteSurface(square1)

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(1, [3], name="FLUX_SQUARE_TOP")
    gmsh.model.addPhysicalGroup(1, [1,2,4], name="PRESSURE_UNIFORM_0")
    gmsh.model.addPhysicalGroup(2, [square1], name="DOMAIN")

    # for possible square meshes
    if not params.triangle_mesh:
        gmsh.option.setNumber("Mesh.RecombineAll", 1)

    # generate a 3D mesh
    gmsh.model.mesh.generate(2)
    
    # save as a .med file
    med_file = params.mesh_file + ".med"
    gmsh.write(med_file)
    
    # close gmsh
    gmsh.finalize()
    
    # translate .med file to a format readable by MoFEM and assign values to physical groups
    h5m_file=params.mesh_file + ".h5m"    
    !{read_med} -med_file {med_file} -output_file {h5m_file} -meshsets_config {params.config_file} -dim 2 -adj_dim 1 -log_sl error
    
    # visualise the mesh
    if params.show_mesh:
        vtk_file=params.mesh_file + ".vtk"
        !mbconvert {h5m_file} {vtk_file}

        mesh = pv.read(vtk_file)
        mesh = mesh.shrink(0.98)

        p = pv.Plotter(notebook=True)
        p.add_mesh(mesh, smooth_shading=False)

        p.camera_position = "xy"
        p.show(jupyter_backend='ipygany')
    
    return


## Square Sin Cos analysis


In [ ]:
# generation of a config file - what attributes should the blocksets have
def generateConfig_squareSinCos(params):
    # Open the file for writing
    with open(params.config_file, 'w') as f:
        # FLUX_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_FLUX_SQUARE_SINCOS]', 'number_of_attributes=1', 'user1='+str(params.conductivity), ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_PRESSURE_SQUARE_SINCOS]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # SOURCE_SQUARE_SINCOS
        data = ['[SET_ATTR_SOURCE_SQUARE_SINCOS]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)

In [ ]:
# gmsh creation of a 3D beam + visualisation of it
def generateMesh_squareSinCos(params):
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)

    square1 = gmsh.model.occ.add_rectangle(0, 0, 0, params.length_x, params.length_y)

    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.occ.synchronize()

    # # ensuring a structured mesh with required element size 
    N = int(params.length_x / params.element_size) + 1

    for n in range(len(gmsh.model.getEntities(1))):
        gmsh.model.mesh.setTransfiniteCurve(n+1, N,'Progression', 1.0)

    gmsh.model.mesh.setTransfiniteSurface(square1)

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(1, [3], name="FLUX_SQUARE_SINCOS")
    gmsh.model.addPhysicalGroup(1, [1,2,4], name="PRESSURE_SQUARE_SINCOS")
    gmsh.model.addPhysicalGroup(2, [square1], name="SOURCE_SQUARE_SINCOS")

    # for possible square meshes
    if not params.triangle_mesh:
        gmsh.option.setNumber("Mesh.RecombineAll", 1)

    # generate a 3D mesh
    gmsh.model.mesh.generate(2)
    
    # save as a .med file
    med_file = params.mesh_file + ".med"
    gmsh.write(med_file)
    
    # close gmsh
    gmsh.finalize()
    
    # translate .med file to a format readable by MoFEM and assign values to physical groups
    h5m_file=params.mesh_file + ".h5m"    
    !{read_med} -med_file {med_file} -output_file {h5m_file} -meshsets_config {params.config_file} -dim 2 -adj_dim 1 -log_sl error
    
    # visualise the mesh
    if params.show_mesh:
        vtk_file=params.mesh_file + ".vtk"
        !mbconvert {h5m_file} {vtk_file}

        mesh = pv.read(vtk_file)
        mesh = mesh.shrink(0.98)

        p = pv.Plotter(notebook=True)
        p.add_mesh(mesh, smooth_shading=False)

        p.camera_position = "xy"
        p.show(jupyter_backend='ipygany')
    
    return


## Square NONLINEAR Sin Cos analysis

In [ ]:
# generation of a config file - what attributes should the blocksets have
def generateConfig_squareNonlinearSinCos(params):
    # Open the file for writing
    with open(params.config_file, 'w') as f:
        # FLUX_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_FLUX_SQUARE_NONLINEAR_SINCOS]', 'number_of_attributes=1', 'user1='+str(params.conductivity), ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_PRESSURE_SQUARE_SINCOS]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # SOURCE_SQUARE_SINCOS
        data = ['[SET_ATTR_SOURCE_SQUARE_NONLINEAR_SINCOS]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)

In [ ]:
# gmsh creation of a 3D beam + visualisation of it
def generateMesh_squareNonlinearSinCos(params):
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)

    square1 = gmsh.model.occ.add_rectangle(0, 0, 0, params.length_x, params.length_y)

    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.occ.synchronize()

    # # ensuring a structured mesh with required element size 
    N = int(params.length_x / params.element_size) + 1

    for n in range(len(gmsh.model.getEntities(1))):
        gmsh.model.mesh.setTransfiniteCurve(n+1, N,'Progression', 1.0)

    gmsh.model.mesh.setTransfiniteSurface(square1)

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(1, [3], name="FLUX_SQUARE_NONLINEAR_SINCOS")
    gmsh.model.addPhysicalGroup(1, [1,2,4], name="PRESSURE_SQUARE_SINCOS")
    gmsh.model.addPhysicalGroup(2, [square1], name="SOURCE_SQUARE_NONLINEAR_SINCOS")

    # for possible square meshes
    if not params.triangle_mesh:
        gmsh.option.setNumber("Mesh.RecombineAll", 1)

    # generate a 3D mesh
    gmsh.model.mesh.generate(2)
    
    # save as a .med file
    med_file = params.mesh_file + ".med"
    gmsh.write(med_file)
    
    # close gmsh
    gmsh.finalize()
    
    # translate .med file to a format readable by MoFEM and assign values to physical groups
    h5m_file=params.mesh_file + ".h5m"    
    !{read_med} -med_file {med_file} -output_file {h5m_file} -meshsets_config {params.config_file} -dim 2 -adj_dim 1 -log_sl error
    
    # visualise the mesh
    if params.show_mesh:
        vtk_file=params.mesh_file + ".vtk"
        !mbconvert {h5m_file} {vtk_file}

        mesh = pv.read(vtk_file)
        mesh = mesh.shrink(0.98)

        p = pv.Plotter(notebook=True)
        p.add_mesh(mesh, smooth_shading=False)

        p.camera_position = "xy"
        p.show(jupyter_backend='ipygany')
    
    return
